# Desafio Python

## Etapa 1

### Importação das bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

## Etapa 2

### Leitura do dataset e remoção de linhas duplicadas

In [ ]:
df = pd.read_csv('googleplaystore.csv')
df = df.drop_duplicates()

### 5 aplicativos com maior número de instalações

In [ ]:
# Limpa a coluna 'Installs' removendo ',' e '+' e converte para número
df['Installs'] = df['Installs'].astype(str).str.replace(',', '').str.replace('+', '')
df['Installs'] = pd.to_numeric(df['Installs'], errors='coerce')

# Remove duplicatas mantendo apenas a versão com mais instalações
df_sem_duplicados = df.sort_values(by='Installs', ascending=False).drop_duplicates(subset='App', keep='first')

# Seleciona os 5 apps mais instalados
cinco_apps_mais_instalados = df_sem_duplicados.head(5)

# Exibe gráfico de barras com os dados
plt.figure(figsize=(10,6))
plt.bar(cinco_apps_mais_instalados['App'], cinco_apps_mais_instalados['Installs'], color='skyblue')
plt.title('5 Aplicativos Mais Instalados')
plt.xlabel('Aplicativo')
plt.ylabel('Número de Instalações')
plt.tight_layout()
plt.show()

### Gráfico de pizza com as categorias de apps e frequência de ocorrência.

In [ ]:
# Conta a quantidade de apps por categoria
contagem_categori = df['Category'].value_counts()

# Gráfico de pizza das categorias
plt.figure(figsize=(35, 35))
plt.pie(
    contagem_categori,
    labels=contagem_categori.index,
    autopct='%1.1f%%',
    startangle=140,
    textprops={'fontsize': 18}
)
plt.axis('equal')
plt.tight_layout()
plt.show()


### Aplicativo mais caro

In [ ]:
# Remove o símbolo "$" na coluna 'Price'
df['Price'] = df['Price'].astype(str).str.replace('$', '')

# Garante que a coluna 'Price' é numérica
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Encontra o app mais caro
app_mais_caro = df[['App', 'Price']].loc[df['Price'].idxmax()]

# Imprime resultados
print(f"Nome: {app_mais_caro['App']}")
print(f"Preço: {app_mais_caro['Price']}$")

### Número de apps classificados como "Mature 17+

In [ ]:
# Conta quantos apps têm classificação 'Mature 17+'
num_mature = df['Content Rating'].value_counts().get('Mature 17+', 0)

# Exibe o resultado
print(f"Número de apps classificados como 'Mature 17+': {num_mature}")


### Número de reviews dos 10 apps mais avaliados

In [ ]:
# Garante que a coluna 'Reviews' é numérica
df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')

# Remove apps duplicados
df_sem_duplicados = df.sort_values(by='Reviews', ascending=False).drop_duplicates(subset='App', keep='first')

# Seleciona os 10 apps com mais reviews
dez_app_mais_reviews = df_sem_duplicados[['App', 'Reviews']].sort_values(by='Reviews', ascending=False).head(10)

# Imprime resultados
for i, row in dez_app_mais_reviews.iterrows():
    print(f"Nome: {row['App']} | Número de reviews: {int(row['Reviews']):,} \n")

### Proporção número de apps gratis e apps pagos

In [ ]:
df['Type'] = df['Type'].astype(str).str.strip()
df = df[df['Type'].isin(['Free', 'Paid'])]

contagem_tipos = df['Type'].value_counts()
print(f"Contagem de apps gratuitos e pagos:\n{contagem_tipos}")

In [ ]:
# Cria gráfico de pizza
plt.figure(figsize=(6, 6))
plt.pie(
    contagem_tipos,
    labels=contagem_tipos.index,
    autopct='%1.1f%%',
    startangle=100,
    colors=['lightgreen', 'lightsalmon']
)
plt.title('Proporção de Apps Gratuitos e Pagos')
plt.axis('equal')
plt.tight_layout()
plt.show()

### 5 categorias com maior número de apps

In [ ]:
# Conta número de apps por categoria
apps_por_categoria = df['Category'].value_counts()
cinco_categorias_mais_apps = apps_por_categoria.head(5)

# Imprime resultados
print("5 categorias com mais apps:")
print(cinco_categorias_mais_apps.head(5))

In [ ]:
# Cria gráfico de barras
plt.figure(figsize=(8, 5))
cinco_categorias_mais_apps.plot(kind='bar', color='skyblue')

plt.title('5 Categorias com Mais Aplicativos')
plt.ylabel('Quantidade de Apps')
plt.xlabel('Categoria')
plt.xticks(rotation=0)

# Adiciona valores em cima das barras
for i, valor in enumerate(cinco_categorias_mais_apps.values):
    plt.text(i, valor + 2, str(valor), ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## Etapa 3

In [ ]:
# Cria uma conexão com o banco de dados SQLite
engine = create_engine('sqlite:///apps_google_play.db')

# Salva o DataFrame
df.to_sql('apps', con=engine, if_exists='replace', index=False)

### Nome e a categoria dos 10 apps mais instalados.

In [ ]:
query_a = """
SELECT App, Category
FROM apps
ORDER BY Installs DESC
LIMIT 10
"""
resultado = pd.read_sql_query(query_a, con=engine)
print(resultado)


### Nome dos apps classificados como Mature 17+

In [ ]:
# Consulta os apps com classificação 'Mature 17+'
query = """
SELECT App
FROM apps
WHERE `Content Rating` = 'Mature 17+'
"""

# Executa a consulta e carrega o resultado em um DataFrame
resultado = pd.read_sql_query(query, engine)

# Exibir os resultados
print(resultado) 

### Número médio de reviews por categoria

In [ ]:
# Consulta a média de avaliações por categoria
query = """
SELECT Category, AVG(Reviews) AS media_reviews
FROM apps
GROUP BY Category
ORDER BY media_reviews DESC
"""
# Exibir os resultados
resultado = pd.read_sql_query(query, con=engine)
print(resultado)

# Gráfico de barras com a média de avaliações por categoria
plt.figure(figsize=(12, 6))
plt.bar(resultado['Category'], resultado['media_reviews'], color='skyblue')
plt.xticks(rotation=90)  # Rotaciona os rótulos do eixo X para melhor leitura
plt.xlabel('Categoria')
plt.ylabel('Média de Avaliações')
plt.title('Média de Avaliações por Categoria de App')
plt.tight_layout()
plt.show()


### Apps com preço maior que 20$

In [ ]:
# Consulta apps com preço acima de 20, em ordem decrescente
query_d = """
SELECT App, Price
FROM apps
WHERE Price > 20
ORDER BY Price DESC
"""
resultado = pd.read_sql_query(query_d, con=engine)
print(resultado)

# Ordena por preço crescente e remove caracteres não-ASCII dos nomes dos apps
resultado = resultado.sort_values(by='Price', ascending=True)
resultado['App'] = resultado['App'].str.encode('ascii', 'ignore').str.decode('ascii')

# Cria gráfico de barras horizontais
plt.figure(figsize=(10, 8))
plt.barh(resultado['App'], resultado['Price'], color='orange')
plt.xlabel('Preço')
plt.title('Apps com Preço Acima de 20$')
plt.tight_layout()
plt.show()
